# SYK VQE with KAN-based Optimizer

This notebook demonstrates replacing the classical optimizer inside VQE with a small KAN (neural network) that proposes parameter updates. Use this as an experimental comparator vs COBYLA/SLSQP.

**Warning:** This is experimental and for small toy Hamiltonians only.

In [ ]:
!pip install qiskit qiskit-ibm-provider numpy scipy matplotlib torch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from qiskit import Aer, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
algorithm_globals.random_seed = 42
np.random.seed(42)
torch.manual_seed(42)


## Build toy Hamiltonian (same as previous notebooks)

In [ ]:
from qiskit.quantum_info import Pauli
pauli_list = [
    ( -1.2, "ZZ"),
    (  0.6, "XI"),
    (  0.4, "IZ"),
    (  0.2, "YI")
]
H = SparsePauliOp.from_list([(p, c) for c, p in pauli_list])
dense_H = H.to_matrix()
eigvals, eigvecs = np.linalg.eigh(dense_H)
print('Exact ground energy:', eigvals[0])
n_qubits = H.num_qubits


## MERA ansatz (reuse small ansatz)

In [ ]:
from qiskit.circuit import ParameterVector, QuantumCircuit
def small_ansatz(n_qubits, reps=2):
    params = ParameterVector('t', n_qubits*2*reps)
    qc = QuantumCircuit(n_qubits)
    idx = 0
    for r in range(reps):
        for q in range(n_qubits):
            qc.ry(params[idx], q); idx+=1
            qc.rz(params[idx], q); idx+=1
        for q in range(n_qubits-1):
            qc.cz(q, q+1)
    return qc, params

ansatz, params = small_ansatz(n_qubits, reps=2)
ansatz.draw('mpl')


## KAN surrogate proposer

We create a small neural network that, given the current parameter vector and recent energies, proposes a new parameter vector. We'll train it online to minimize measured ⟨H⟩. This is a simple proof-of-concept.

In [ ]:
class KANProposer(nn.Module):
    def __init__(self, param_dim, hidden=128):
        super().__init__()
        self.net = nn.Sequential(            nn.Linear(param_dim+1, hidden),            nn.ReLU(),            nn.Linear(hidden, hidden),            nn.ReLU(),            nn.Linear(hidden, param_dim)        )
    def forward(self, params, energy):
        x = torch.cat([params, energy.unsqueeze(0)], dim=0)
        return self.net(x)

# initialize
param_dim = len(params)
proposer = KANProposer(param_dim)
opt_proposer = torch.optim.Adam(proposer.parameters(), lr=1e-3)


## VQE loop with KAN proposer (simulator statevector for energy eval)

Loop: propose new params -> evaluate ⟨H⟩ via statevector simulation -> compute loss = energy and backprop through proposer.

In [ ]:
backend = Aer.get_backend('aer_simulator_statevector')

def energy_from_params(param_values):
    qc_bound = ansatz.bind_parameters(param_values)
    sv = Statevector.from_instruction(qc_bound)
    psi = sv.data
    e = np.vdot(psi.conj(), dense_H @ psi).real
    return float(e)

# warm-start: random initial params
current_params = np.random.normal(0, 0.1, size=param_dim).astype(np.float32)
for epoch in range(50):
    p_tensor = torch.tensor(current_params, dtype=torch.float32)
    energy_tensor = torch.tensor([energy_from_params(current_params)], dtype=torch.float32)
    proposed = proposer(p_tensor, energy_tensor)
    alpha = 0.2
    new_params = (current_params + alpha * proposed.detach().numpy()).astype(np.float32)
    e_curr = energy_from_params(current_params)
    e_new = energy_from_params(new_params)
    loss = torch.tensor(e_new, requires_grad=True)
    opt_proposer.zero_grad()
    loss.backward()
    opt_proposer.step()
    if e_new < e_curr:
        current_params = new_params
    if epoch % 5 == 0:
        print(f'Epoch {epoch}: e_curr={e_curr:.6f}, e_new={e_new:.6f}')
print('Final energy (KAN):', energy_from_params(current_params))
print('Exact ground:', eigvals[0])


## Comparison with COBYLA (baseline)

In [ ]:
# baseline using VQE with COBYLA for comparison
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
qi = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))
vqe = VQE(ansatz=ansatz, optimizer=COBYLA(maxiter=200), quantum_instance=qi)
res = vqe.compute_minimum_eigenvalue(SparsePauliOp.from_list([(p,c) for c,p in pauli_list]))
print('COBYLA VQE energy:', res.eigenvalue.real)
